# **Fine Tuning MobileBERT**

In [25]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

from transformers import MobileBertForSequenceClassification, MobileBertTokenizer,\
    Trainer, TrainingArguments

from datasets import Dataset

from sklearn.metrics import f1_score, accuracy_score

In [3]:
# Load Preprocessed Data 

train_df = pd.read_csv(os.path.join('data', 'phishing_dataset', 'train.csv'))
test_df = pd.read_csv(os.path.join('data', 'phishing_dataset', 'test.csv'))

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [4]:
# Load tokenizer

tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=512, truncation=True, 
                     padding="max_length", add_special_tokens=True, return_tensors = 'pt')

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)



Map: 100%|██████████| 7047/7047 [00:18<00:00, 371.04 examples/s]


In [30]:
# Load model & train

model = MobileBertForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=2)

from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(preds):
    out = np.argmax(preds.predictions, axis=1)
    return {
        'accuracy': accuracy_score(preds.label_ids, out),
        'f1': f1_score(preds.label_ids, out, average='weighted')  # You can change the average type as needed
    }



training_args = TrainingArguments(
    output_dir = os.path.join('models', 'checkpoints'), 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_steps = 1
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


100%|██████████| 1410/1410 [03:02<00:00,  7.73it/s]
Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
e:\GitHub\llm-sms-phishing-detection\venv\lib\site-packages\datasets\load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [6]:
trainer.train()

  5%|▌         | 500/9864 [01:12<23:17,  6.70it/s]Checkpoint destination directory models\checkpoints\checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 122154.52, 'learning_rate': 1.8986212489862126e-05, 'epoch': 0.15}


 10%|█         | 1000/9864 [02:25<27:39,  5.34it/s]Checkpoint destination directory models\checkpoints\checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'loss': 1.7589, 'learning_rate': 1.7972424979724253e-05, 'epoch': 0.3}


 15%|█▌        | 1500/9864 [03:46<23:42,  5.88it/s]Checkpoint destination directory models\checkpoints\checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

 96%|█████████▋| 9500/9864 [25:06<00:51,  7.06it/s]

{'loss': 0.0487, 'learning_rate': 7.380373073803731e-07, 'epoch': 2.89}


                                                   
100%|██████████| 9864/9864 [27:01<00:00,  6.08it/s]

{'eval_loss': 0.1447024643421173, 'eval_accuracy': 0.9824038597984958, 'eval_runtime': 59.7336, 'eval_samples_per_second': 117.974, 'eval_steps_per_second': 23.605, 'epoch': 3.0}
{'train_runtime': 1621.7974, 'train_samples_per_second': 30.411, 'train_steps_per_second': 6.082, 'train_loss': 6193.007407180295, 'epoch': 3.0}


TrainOutput(global_step=9864, training_loss=6193.007407180295, metrics={'train_runtime': 1621.7974, 'train_samples_per_second': 30.411, 'train_steps_per_second': 6.082, 'train_loss': 6193.007407180295, 'epoch': 3.0})

In [43]:
checkpoint = "models\\checkpoints\\checkpoint-9500"

model = MobileBertForSequenceClassification.from_pretrained(checkpoint)


trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics = compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.evaluate()



100%|██████████| 1410/1410 [00:59<00:00, 23.70it/s]


{'eval_loss': 0.16192379593849182,
 'eval_accuracy': 0.9816943380161771,
 'eval_f1': 0.9816642980997267,
 'eval_runtime': 59.7833,
 'eval_samples_per_second': 117.876,
 'eval_steps_per_second': 23.585}

In [ ]:
# Evaluation Details

# TODO


In [52]:
# Try custom sample

txt1 = """

Dear Luke Ingram

Thank you for starting your application for our Data Science-Machine Learning Engineer position at Zynex Medical . We noticed that you have not fully completed your application.

You may continue the application process by navigating to the following page: Please Click Here to Complete Your Application

We hope you consider completing and submitting your application before the position is no longer available.

Thank you for your interest in our company, we look forward to seeing your completed application!
"""

txt2 = "Amazon here: click here to reset your passowrd www.samazon.com/passwordreset"

txt3 = "Check out this profile in linkedin: www.linkedin.com/person333"

tokens = tokenizer.encode_plus(txt3, max_length=512, truncation=True, padding="max_length", add_special_tokens=True, 
                               return_tensors='pt')

tokens



{'input_ids': tensor([[  101,  4638,  2041,  2023,  6337,  1999,  5799,  2378,  1024,  7479,
          1012,  5799,  2378,  1012,  4012,  1013,  2711, 22394,  2509,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [53]:
tokens = tokens.to(torch.device('cuda'))
output = model(**tokens)

probs = F.softmax(output[0], dim=-1)

probs

tensor([[1.0000e+00, 9.3530e-09]], device='cuda:0', grad_fn=<SoftmaxBackward0>)